# COVID-19

### Data on COVID-19 (coronavirus) by Our World in Data

Datasource:<br/>
[https://github.com/owid/covid-19-data/tree/master/public/data/](https://github.com/owid/covid-19-data/tree/master/public/data/)

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt
from ipywidgets import widgets, interactive

%matplotlib widget

In [ ]:
path = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
df = pd.read_csv(path)
df['date'] = df['date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))

In [ ]:
loc_selection = widgets.SelectMultiple(options = ['All'] + df['location'].unique().tolist(), 
                                       value=['Norway', 'Poland', 'Germany'], 
                                       description='Select country', 
                                       rows=10)
start_selection = widgets.DatePicker(value=dt.datetime(2020, 7, 1), description='Start')
end_selection = widgets.DatePicker(value=df['date'].max(), description='End')

def plot_ovid(location, start_time, end_time):
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)
    columns = ['date', 'location', 'new_cases', 'population']
    time_idx = (df['date']>=start_time-dt.timedelta(days=14)) & (df['date']<=end_time)
    if 'All' in location:
        df_plt = df[time_idx][columns].set_index('date')
    else:
        df_plt = df[(time_idx) & (df['location'].isin(location))][columns].set_index('date')
    df_plt['cc'] = df_plt['new_cases'].rolling(14).sum()
    df_plt['cpt'] = df_plt['cc']*1e5/df_plt['population']
    plt.close()
    fig, ax = plt.subplots()
    df_plt.groupby('location')['cpt'].plot()
    ax.set_xlim((start_time, dt.datetime.now()))
    ax.set_ylim((0, 400))
    ax.legend()
    ax.grid()
    ax.set_title('biweekly new cases per 10k')
    plt.show()
    return fig, ax

In [ ]:
interactive(plot_ovid, location=loc_selection, start_time=start_selection, end_time=end_selection)